In [1]:
from sklearn.cluster import KMeans
from scipy.spatial import distance
import numpy as np
import pandas as pd
import random
import scipy.io as scio
from numpy import genfromtxt
from KmeansPPcenters import KMeanPlusPlus
from KMeansOut import kmeansOutliers, cost,  compute_phi_star
from sklearn.metrics import average_precision_score, precision_recall_curve
from LO import add_noise, KPP_centers, random_centers, LO_cost, LloydOut, KMO_centers

In [2]:
read_data = genfromtxt('realDataProcessed/skin.csv', delimiter=',')
print("data loaded")
skin_labels= read_data[:,3]
skin_data=read_data[:,0:3]

num_clusters=[2, 3]
zs =[4, 5]
min_value= 210
max_value=255
#min_values=[-16, 0, 7.693475e-08]
#max_values= [15, 20, 33]
tol= .05
itr=1000

data= skin_data[0:20:,]
data

data loaded


array([[ 74.,  85., 123.],
       [ 73.,  84., 122.],
       [ 72.,  83., 121.],
       [ 70.,  81., 119.],
       [ 70.,  81., 119.],
       [ 69.,  80., 118.],
       [ 70.,  81., 119.],
       [ 70.,  81., 119.],
       [ 76.,  87., 125.],
       [ 76.,  87., 125.],
       [ 77.,  88., 126.],
       [ 77.,  88., 126.],
       [ 77.,  88., 126.],
       [ 78.,  89., 127.],
       [ 77.,  85., 125.],
       [ 78.,  86., 126.],
       [ 78.,  86., 126.],
       [ 77.,  85., 125.],
       [ 76.,  84., 124.],
       [ 77.,  85., 125.]])

In [4]:
def LO_cost(data_inliers, cid, centers, z):
    data_copy= data.copy()
    dist= distance.cdist(data_copy, np.array(centers))
    dist = np.amin(dist, axis = 1)
    indx_list = np.argpartition(dist, -z)[-z:] #get index of farthest z points
    data_new = np.delete(data_copy, indx_list, axis=0)
    cid_pruned = cid.copy()
    cid_pruned[indx_list] = len(centers) + 1 # comment this line out if you do not want to remove points
    cost= np.zeros(len(centers))

    for i in range(len(centers)):
        cluster_indx = np.where(cid_pruned==i)
        cluster_points = data[cluster_indx]
        cost[i] = np.sum((cluster_points-centers[i])**2)
    total_cost= np.sum(cost)/len(data_new)

    return total_cost, indx_list

In [10]:
data_copy= data.copy()
kpp_centers = KPP_centers(data_with_outliers, num_cluster)
dist=distance.cdist(data_copy, np.array(kpp_centers))
dist = np.amin(dist, axis = 1)
indx_list = np.argpartition(dist, -z)[-z:] #get index of farthest z points
data_new = np.delete(data_copy, indx_list, axis=0)
centers, cid, indx_list, precision, recall, data_out=LloydOut(data_with_outliers, kpp_centers, num_cluster, z, tol, itr, z_indx )
cid_pruned = cid.copy()
cid_pruned[indx_list] = len(centers) + 1 # comment this line out if you do not want to remove points
cost= np.zeros(len(centers))

In [3]:
for num_cluster in num_clusters:
    for z in zs:
        print("num_cluster:{}, z:{}".format(num_cluster, z))
        print("Data before:{}".format(data))
        data_with_outliers, z_indx, data_inliers = add_noise(skin_data[0:20:,], z, min_value, max_value)
        print("Data:{}".format(data))
        print("Data With Outliers:{}".format(data_with_outliers))
        kpp_centers = KPP_centers(data_with_outliers, num_cluster)
        centers, cid, indx_list, precision, recall, data_out=LloydOut(data_with_outliers, kpp_centers, num_cluster, z, tol, itr, z_indx )
        print("KPP LO done")
        print("Data:{}".format(data))
        print("Data With Outliers:{}".format(data_with_outliers))
        rand_centers= random_centers(data_with_outliers, num_cluster)
        centers, cid, indx_list, precision, recall, data_out= LloydOut(data_with_outliers, rand_centers, num_cluster, z, tol, itr, z_indx)
        print("Random LO done")
        print("Data:{}".format(data))
        print("Data With Outliers:{}".format(data_with_outliers))
        phi_star= compute_phi_star(data, num_cluster, kpp_centers, z)
        kmo_centers= KMO_centers(data_with_outliers, num_cluster, 10000*phi_star, z)
        centers, cid, indx_list, precision, recall, data_out= LloydOut(data_with_outliers, kmo_centers, num_cluster, z, tol, itr, z_indx)

num_cluster:2, z:4
Data before:[[ 74.  85. 123.]
 [ 73.  84. 122.]
 [ 72.  83. 121.]
 [ 70.  81. 119.]
 [ 70.  81. 119.]
 [ 69.  80. 118.]
 [ 70.  81. 119.]
 [ 70.  81. 119.]
 [ 76.  87. 125.]
 [ 76.  87. 125.]
 [ 77.  88. 126.]
 [ 77.  88. 126.]
 [ 77.  88. 126.]
 [ 78.  89. 127.]
 [ 77.  85. 125.]
 [ 78.  86. 126.]
 [ 78.  86. 126.]
 [ 77.  85. 125.]
 [ 76.  84. 124.]
 [ 77.  85. 125.]]
Data:[[ 74.  85. 123.]
 [ 73.  84. 122.]
 [ 72.  83. 121.]
 [ 70.  81. 119.]
 [ 70.  81. 119.]
 [ 69.  80. 118.]
 [ 70.  81. 119.]
 [ 70.  81. 119.]
 [ 76.  87. 125.]
 [ 76.  87. 125.]
 [ 77.  88. 126.]
 [ 77.  88. 126.]
 [ 77.  88. 126.]
 [ 78.  89. 127.]
 [ 77.  85. 125.]
 [ 78.  86. 126.]
 [ 78.  86. 126.]
 [ 77.  85. 125.]
 [ 76.  84. 124.]
 [ 77.  85. 125.]]
Data With Outliers:[[-137.68683116  325.24398899  362.59058606]
 [  73.           84.          122.        ]
 [  72.           83.          121.        ]
 [  70.           81.          119.        ]
 [  70.           81.          119.        